# Build an Agent with AgentExecutor (Legacy)


https://python.langchain.com/v0.2/docs/how_to/agent_executor/

By themselves, language models can't take actions - they just output text. 

A big use case for LangChain is creating agents. 

Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be. 


The results of those actions can then be fed back into the agent and it determines whether more actions are needed, 

or whether it is okay to finish.

# Info
This section will cover building with the legacy LangChain AgentExecutor. 

These are fine for getting started, but past a certain point, we will likely want flexibility and control that they do not offer. 

For working with more advanced agents, it's recommended checking out LangGraph Agents or the migration guide

Langgraph: https://python.langchain.com/v0.2/docs/concepts/#langgraph

Migration FROM LangChain Agents to LangGraph : https://python.langchain.com/v0.2/docs/how_to/migrate_agent/


In this tutorial, we will build an agent that can interact with multiple different tools: 
1. one being a local database, 
2. the other being a search engine. 

We will be able to ask this agent questions, watch it call tools, and have conversations with it.

# Concepts
Concepts we will cover are:

1. Using language models, in particular their tool calling ability
2. Creating a Retriever to expose specific information to our agent
3. Using a Search Tool to look up things online
4. Chat History, which allows a chatbot to "remember" past interactions and take them into account when responding to follow-up questions.
5. Debugging and tracing your application using LangSmith

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

# Define tools
We first need to create the tools we want to use. 

We will use two tools: ```Tavily``` (to search online) and then a ```retriever``` over a local index we will create



# Tavily

We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1725284938, 'localtime': '2024-09-02 06:48'}, 'current': {'last_updated_epoch': 1725284700, 'last_updated': '2024-09-02 06:45', 'temp_c': 17.8, 'temp_f': 64.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 270, 'wind_dir': 'W', 'pressure_mb': 1014.0, 'pressure_in': 29.95, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 75, 'feelslike_c': 17.8, 'feelslike_f': 64.0, 'windchill_c': 14.2, 'windchill_f': 57.6, 'heatindex_c': 14.3, 'heatindex_f': 57.8, 'dewpoint_c': 12.3, 'dewpoint_f': 54.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 7.3, 'gust_kph': 11.7}}"},
 {'url': 'https://www.wunderground.com/hourly/

# Retriever

We will also create a retriever over some data of our own. 




In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [8]:
retriever.invoke("how to upload a dataset")[0]

Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here  

In [13]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [14]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

# Create List of Tools:

Now that we have created both, we can create a list of tools that we will use downstream.



In [15]:
tools = [search, retriever_tool]

# LLM

In [18]:

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

In [20]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

# Bind the tools to LLM

In [21]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the content field as well as the tool_calls field.



In [22]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


Now, let's try calling it with some input that would expect a tool to be called.



In [23]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_RdAJVp3kJNv0ffRapA4ugh1W', 'type': 'tool_call'}]


# Create the agent
Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent - for more information on this type of agent, as well as other options, see

https://python.langchain.com/v0.2/docs/concepts/#agent_types/

If you want to see the contents of this prompt and have access to LangSmith, you can go to:

https://smith.langchain.com/hub/hwchase17/openai-functions-agent

In [25]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

# Initialize the agent

Now, we can initalize the agent with the LLM, the prompt, and the tools. 

The agent is responsible for taking in input and deciding what actions to take. 
Crucially, the Agent does not execute those actions - that is done by the AgentExecutor (next step). 

For more information about how to think about these components, see our conceptual guide.

# Important Note:
Note that we are passing in the ```model```, not ```model_with_tools```. That is because create_tool_calling_agent will call ```.bind_tools``` for us under the hood.

In [27]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

In [28]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

# Run the agent

We can now run the agent on a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions).

First up, let's how it responds when there's no need to call a tool:

In [29]:
agent_executor.invoke({"input": "hi!"})

{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the LangSmith trace

https://smith.langchain.com/public/8441812b-94ce-4832-93ec-e1114214553a/r

Let's now try it out on an example where it should be invoking the retriever



In [31]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith can significantly assist with testing by providing a platform for logging and evaluating the performance of machine learning applications, particularly those built with large language models (LLMs). Here are some key ways it helps with testing:\n\n1. **Monitoring and Evaluation**: LangSmith enables developers to closely monitor their applications. This includes tracking the performance of various models in real-time to ensure they are functioning as expected.\n\n2. **Trace Logging**: It allows developers to log traces of their application's execution, which helps in diagnosing issues and understanding model behavior during testing.\n\n3. **Dataset Management**: Users can create and manage datasets that serve as test cases, enabling structured testing of application outputs against expected results.\n\n4. **Custom Evaluators**: LangSmith supports the creation of custom evaluators, allowing developers to define speci

Let's take a look at the LangSmith trace to make sure it's actually calling that.

https://smith.langchain.com/public/762153f6-14d4-4c98-8659-82650f860c62/r

Now let's try one where it needs to call the search tool:



In [32]:
agent_executor.invoke({"input": "whats the weather in sf?"})

{'input': 'whats the weather in sf?',
 'output': 'The current weather in San Francisco is as follows:\n\n- **Temperature**: 17.8°C (64.0°F)\n- **Condition**: Partly cloudy\n- **Wind**: 6.9 mph from the west\n- **Humidity**: 87%\n- **Visibility**: 16 km (9 miles)\n\nFor more detailed forecasts, you can check sources like [WeatherAPI](https://www.weatherapi.com/) or [Weather Underground](https://www.wunderground.com/hourly/us/ca/san-francisco/date/2024-09-02).'}

Let's take a look at the LangSmith trace to make sure it's actually calling that.

https://smith.langchain.com/public/36df5b1a-9a0b-4185-bae2-964e1d53c665/r